In [13]:
import os
import torch
import torch.nn as nn
from torchvision import transforms

In [14]:
import sys
sys.path.insert(1, "../../../")

In [15]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.yolov8cls_path import Model

In [16]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [17]:
data_path = '../../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [18]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [19]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [20]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

# Inference

In [21]:
model = Model(num_classes=10, 
              residual_connection=False, 
              CSP=False, 
              add_hidden=False,
              classifyV8=False,
              bottleneck=1.0, 
              variant='s', 
              device=device, 
              dtype=torch.float32)

In [22]:
img = train_dataset[0][0].unsqueeze(0)

In [23]:
from torch.profiler import profile, ProfilerActivity

In [24]:
prof = profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
                                    schedule=torch.profiler.schedule(wait=1, warmup=1, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../../log/YOLOv8cls-version-1/inference'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True)

prof.start()
for i in range(12):
    out = model(img)
    prof.step()
prof.stop()
print(prof.key_averages().table(row_limit=-1))

--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                            Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total GFLOPs  
--------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   ProfilerStep*         3.33%      12.511ms       100.00%     376.182ms      37.618ms       3.57 Mb    -292.58 Mb            10            --  
                      Conv block        11.06%      41.615ms        95.01%     357.405ms       1.430ms     296.15 Mb           0 b           250            --  
                    aten::conv2d         0.41%       1.555ms        68.01%     255.833ms     983.972us      98.59 Mb           0 b           260        34.647  
               aten::convolution  